In [1]:
import boto3
import io
from sentence_transformers import SentenceTransformer
import pandas as pd
import os

#from your_segmentation_module import segment_paper  # Replace with your segmentation script
#from your_embedding_module import generate_embedding  # Replace with your embedding script


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from wtpsplit import SaT
sat_sm = SaT("sat-3l-sm")

In [3]:
s3_client = boto3.client('s3')
response = s3_client.get_object(Bucket='arxivpapers', Key='arxiv_papers/0704_0058.txt')
file_content = response['Body'].read().decode('utf-8')  # Adjust encoding as needed

In [67]:

splits_list = []
for split in sat_sm.split(file_content, remove_whitespace_before_inference=True, do_paragraph_segmentation=True):
  splits_list.append(split)

paragraphs_splits = []
for paper in splits_list:
  paras = []
  #print("\nPAPER")
  for paragraph in paper:
    clean_sentences = [sentence.replace("\n", "") for sentence in paragraph]
    para = " ".join(clean_sentences)
    paras.append(para)
  paragraphs_splits.append(paras)




In [36]:
# model = SentenceTransformer("allenai/scibert_scivocab_uncased")
# def generate_embedding():
#     embeddings = model.encode(data)
s3_client = boto3.client('s3')

In [ ]:
# Initialize S3 client
s3_client = boto3.client('s3')


bucket_name = 'arxivpapers'
folders = ['arxiv_papers/' , 'conservapedia_papers/', 'vixra_papers/', 'wiki_papers/']

embedding_output_folder = 'embeddings/' # Output folder for embeddings
model = SentenceTransformer("jordyvl/scibert_scivocab_uncased_sentence_transformer")
def stream_and_process_file(s3_client, bucket_name, key, foldername, embedding_df):
    """
    Stream a file from S3, process it with segmentation and embedding scripts,
    and store the embedding result back in S3.
    """
    # Stream file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=key)
    file_content = response['Body'].read().decode('utf-8')  # Adjust encoding as needed
    
    # DENIZ SEGMENTATION USING SaT
    splits_list = []
    for split in sat_sm.split(file_content, remove_whitespace_before_inference=True, do_paragraph_segmentation=True):
        splits_list.append(split)

    paragraphs_splits = []
    for paper in splits_list:
        paras = []
        #print("\nPAPER")
        for paragraph in paper:
            clean_sentences = [sentence.replace("\n", "") for sentence in paragraph]
            para = " ".join(clean_sentences)
            paras.append(para)
            paragraphs_splits.append(paras)
    print("segmented article")

    #ANYA EMBEDDINGS USING
    embedding = []
    print("calculating embedding for article")
    for split in paragraphs_splits:
        #print('embedding single split')
        embedding.append(model.encode(split))

    print("calculated embeddings for paper")
    #print(embedding[0])
    # Generate embeddings
    # embedding = generate_embedding(segmented_content)
    
    # Save embeddings back to S3
    # embedding_key = f"{embedding_output_folder}{key.split('/')[-1].replace('.txt', '_embedding.json')}"
    # s3_client.put_object(
    #     Bucket=bucket_name,
    #     Key=embedding_key,
    #     Body=str(embedding).encode('utf-8')  # Convert to string or JSON if necessary
    # )
    
    embedding_df = pd.concat([embedding_df, pd.DataFrame({'paper_embeddings': [embedding], 'source': foldername})], ignore_index=True)
    print("added embedding and source to dataframe for this paper_____")
    print(embedding_df.head)
    
    #print(f"Processed and saved embedding for: {key}")

def process_all_files_in_folders(s3_client, bucket_name, folders):
    """
    Iterate over all folders and process files in them with progress tracking.
    """
    total_files = 0
    processed_files = 0

    # First, calculate the total number of .txt files
    for folder in folders:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        total_files += sum(1 for obj in response.get('Contents', []) if obj['Key'].endswith('.txt'))

    print(f"Total files to process: {total_files}")

    # Now process each file with progress tracking
    count = 0
    embedding_df = pd.DataFrame(columns=['paper_embeddings', 'source'])
    for folder in folders:
        print(folder, "paper number: ", count)
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        for obj in response.get('Contents', []):
            file_key = obj['Key']
            if file_key.endswith('.txt'):  # Only process .txt files
                stream_and_process_file(s3_client, bucket_name, file_key, folder, embedding_df)
                processed_files += 1
                # Update progress every 100 files
                if processed_files % 100 == 0 or processed_files == total_files:
                    print(f"Processed {processed_files}/{total_files} files.")
                
    



In [57]:
# Execute the processing
process_all_files_in_folders(s3_client, bucket_name, folders)

Total files to process: 2997
arxiv_papers/ paper number:  0
segmented article
calculating embedding for article
calculated embeddings for paper
added embedding and source to dataframe for this paper_____
<bound method NDFrame.head of                                      paper_embeddings         source
0   [[0.9867997, 0.99300766, 0.7332027, -0.7526853...  arxiv_papers/
1   [[0.7957773, 0.9900172, 0.92319083, 0.62911963...  arxiv_papers/
2   [[0.99640137, 0.9995668, -0.08574731, 0.994866...  arxiv_papers/
3   [[0.9988629, 0.8838065, 0.95620394, -0.9866267...  arxiv_papers/
4   [[0.118258946, 0.89908963, 0.4777597, 0.559316...  arxiv_papers/
..                                                ...            ...
95  [[0.98790735, 0.9912164, 0.99814785, -0.057047...  arxiv_papers/
96  [[0.98058766, 0.99127805, 0.99510354, -0.50756...  arxiv_papers/
97  [[0.82789487, -0.093042366, 0.78423685, -0.600...  arxiv_papers/
98  [[0.82789487, -0.093042366, 0.78423685, -0.600...  arxiv_papers/
99  [[0

KeyboardInterrupt: 

In [63]:
def stream_and_process_file(s3_client, bucket_name, key, foldername, batch):
    """
    Stream a file from S3, process it with segmentation and embedding scripts,
    and add the embedding result to the batch for periodic saving.
    """
    # Stream file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=key)
    file_content = response['Body'].read().decode('utf-8')  # Adjust encoding as needed

    # DENIZ SEGMENTATION USING SaT
    splits_list = []
    for split in sat_sm.split(file_content, remove_whitespace_before_inference=True, do_paragraph_segmentation=True):
        splits_list.append(split)

    paragraphs_splits = []
    for paper in splits_list:
        paras = []
        for paragraph in paper:
            clean_sentences = [sentence.replace("\n", "") for sentence in paragraph]
            para = " ".join(clean_sentences)
            paras.append(para)
            paragraphs_splits.append(paras)

    print("Segmented article")

    # ANYA EMBEDDINGS USING
    embedding = []
    print("Calculating embedding for article")
    for split in paragraphs_splits:
        embedding.append(model.encode(split))

    print("Calculated embeddings for paper")

    # Add the embedding and source to the batch
    batch.append({'paper_embeddings': embedding, 'source': foldername})
    print(f"Added embedding to batch. Current batch size: {len(batch)}")


def save_batch_to_parquet(batch, output_file_prefix, batch_counter):
    """
    Save the current batch to a Parquet file with a batch counter.
    """
    if batch:
        # Convert batch to DataFrame
        batch_df = pd.DataFrame(batch)

        # Generate the output file name based on the batch counter
        output_file = f"{output_file_prefix}_batch{batch_counter}.parquet"

        # Save to Parquet
        batch_df.to_parquet(output_file, index=False, compression='snappy', mode='a')
        print(f"Saved batch {batch_counter} of size {len(batch)} to {output_file}")

        # Clear the batch
        batch.clear()


def process_all_files_in_folders(s3_client, bucket_name, folders, batch_size=100):
    """
    Iterate over all folders and process files in them with periodic batch saving.
    """
    total_files = 0
    processed_files = 0
    batch = []
    output_file = "embeddings/embeddings.parquet"

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # First, calculate the total number of .txt files
    for folder in folders:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        total_files += sum(1 for obj in response.get('Contents', []) if obj['Key'].endswith('.txt'))

    print(f"Total files to process: {total_files}")

    # Process each file
    for folder in folders:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        for obj in response.get('Contents', []):
            file_key = obj['Key']
            if file_key.endswith('.txt'):  # Only process .txt files
                stream_and_process_file(s3_client, bucket_name, file_key, folder, batch)
                processed_files += 1

                # Save the batch periodically
                if len(batch) >= batch_size:
                    save_batch_to_parquet(batch, output_file)

                # Update progress
                if processed_files % 100 == 0 or processed_files == total_files:
                    print(f"Processed {processed_files}/{total_files} files.")

    # Save any remaining data in the final batch
    save_batch_to_parquet(batch, output_file)


In [64]:
# Execute the processing
process_all_files_in_folders(s3_client, bucket_name, folders)

Total files to process: 2997
Segmented article
Calculating embedding for article
Calculated embeddings for paper
Added embedding to batch. Current batch size: 1
Segmented article
Calculating embedding for article
Calculated embeddings for paper
Added embedding to batch. Current batch size: 2
Segmented article
Calculating embedding for article


KeyboardInterrupt: 

START HERE

In [101]:
import boto3
import pandas as pd
from sentence_transformers import SentenceTransformer
from datetime import datetime

# Initialize S3 client
s3_client = boto3.client('s3')

bucket_name = 'arxivpapers'
folders = ['arxiv_papers/', 'conservapedia_papers/', 'vixra_papers/', 'wiki_papers/']

embedding_output_folder = 'embeddings/'  # Output folder for embeddings
model = SentenceTransformer("jordyvl/scibert_scivocab_uncased_sentence_transformer")

def save_batch_to_parquet(batch, output_file_name):
    batch_df = pd.DataFrame(batch)
    batch_df.to_parquet(output_file_name, index=False, compression='snappy')
    print(f"Saved batch to {output_file_name}")

def process_batch_of_papers(s3_client, bucket_name, file_keys, foldername):
    splits_list = []
    # Loop through all papers in the batch (up to 100)
    print("creating batch of papers")
    paper_batch = []
    for key in file_keys:
        # Stream file from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=key)
        file_content = response['Body'].read().decode('utf-8')  # Adjust encoding as needed

        # Append to the batch
        paper_batch.append(file_content)

    print("segmenting papers")
    # Segmentation using SaT for the batch of papers
    for split in sat_sm.split(paper_batch, remove_whitespace_before_inference=True, do_paragraph_segmentation=True):
        print("segmenting w SaT")
        splits_list.append(split)

    print("making paragraph splits")
    paragraphs_splits = []
    # Now clean the sentences and prepare paragraphs
    for paper in splits_list:
        paras = []
        for paragraph in paper:
            clean_sentences = [sentence.replace("\n", "") for sentence in paragraph]
            para = " ".join(clean_sentences)
            paras.append(para)
        paragraphs_splits.append(paras)

    # Embeddings using SentenceTransformer
    print("making embeddings")
    embeddings = []
    for split in paragraphs_splits:
        embeddings.append(model.encode(split))

    print("making batch embeddings dataset")
    # Prepare batch data for saving
    batch_data = [{'paper_embeddings': [embedding], 'source': foldername} for embedding in embeddings]

    # Generate timestamped filename for Parquet output
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_name = f"{embedding_output_folder}/embeddings_{timestamp}.parquet"

    # Save the batch to a Parquet file with timestamp
    print("saving batch dataset to /embeddings")
    save_batch_to_parquet(batch_data, output_file_name)

    print(f"Processed batch of {len(file_keys)} files from {foldername}.")
    return

def process_all_files_in_folders(s3_client, bucket_name, folders, batch_size=100):
    """
    Iterate over all folders, process files in batches, and track progress.
    """
    total_files = 0
    processed_files = 0

    # Calculate the total number of .txt files in all folders
    for folder in folders:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        total_files += sum(1 for obj in response.get('Contents', []) if obj['Key'].endswith('.txt'))

    print(f"Total files to process: {total_files}")

    # Now process the files in batches
    for folder in folders:
        print(f"Processing files from folder: {folder}")
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        file_keys = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.txt')]

        # Process files in batches of 100
        for i in range(0, len(file_keys), batch_size):
            batch_file_keys = file_keys[i:i + batch_size]
            process_batch_of_papers(s3_client, bucket_name, batch_file_keys, folder)
            processed_files += len(batch_file_keys)

            # Update progress
            if processed_files % 100 == 0 or processed_files == total_files:
                print(f"Processed {processed_files}/{total_files} files.")

    print("Finished processing all files.")


In [102]:
process_all_files_in_folders(s3_client, bucket_name, folders[:3])

Total files to process: 2997
Processing files from folder: arxiv_papers/
creating batch of papers
segmenting papers
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT
segmenting w SaT


ArrowInvalid: ('Can only convert 1-dimensional array values', 'Conversion failed for column paper_embeddings with type object')

In [105]:
import boto3
import pandas as pd
from sentence_transformers import SentenceTransformer
from datetime import datetime

# Initialize S3 client
s3_client = boto3.client('s3')

bucket_name = 'arxivpapers'
folders = ['arxiv_papers/', 'conservapedia_papers/', 'vixra_papers/', 'wiki_papers/']

embedding_output_folder = 'embeddings/'  # Output folder for embeddings
model = SentenceTransformer("jordyvl/scibert_scivocab_uncased_sentence_transformer")

def save_batch_to_csv(batch, output_file_name):
    """
    Save the current batch to a CSV file with a timestamp.
    """
    # Convert the batch to a DataFrame
    print("saving batch to csv")
    batch_df = pd.DataFrame(batch)
    
    # Ensure the 'paper_embeddings' column contains lists of floats (not nested arrays)
    batch_df['paper_embeddings'] = batch_df['paper_embeddings'].apply(lambda x: [item.tolist() for item in x])  # Flatten if needed
    
    # Write the DataFrame to a CSV file
    batch_df.to_csv(output_file_name, index=False)
    print(f"Saved batch to {output_file_name}")

def process_batch_of_papers(s3_client, bucket_name, file_keys, foldername):
    splits_list = []
    print("processing batch of papers")
    # Loop through all papers in the batch (up to 100)
    paper_batch = []
    print("creating batch")
    for key in file_keys:
        # Stream file from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=key)
        file_content = response['Body'].read().decode('utf-8')  # Adjust encoding as needed

        # Append to the batch
        paper_batch.append(file_content)
    print("segmenting batch")
    # Segmentation using SaT for the batch of papers
    for split in sat_sm.split(paper_batch, remove_whitespace_before_inference=True, do_paragraph_segmentation=True):
        splits_list.append(split)

    paragraphs_splits = []
    # Now clean the sentences and prepare paragraphs
    for paper in splits_list:
        paras = []
        for paragraph in paper:
            clean_sentences = [sentence.replace("\n", "") for sentence in paragraph]
            para = " ".join(clean_sentences)
            paras.append(para)
        paragraphs_splits.append(paras)

    # Embeddings using SentenceTransformer
    print("embedding batch")
    embeddings = []
    for split in paragraphs_splits:
        embeddings.append(model.encode(split))

    # Prepare batch data for saving
    print("trying to write batch to csv")
    batch_data = [{'paper_embeddings': [embedding], 'source': foldername} for embedding in embeddings]

    # Generate timestamped filename for CSV output
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_name = f"{embedding_output_folder}/embeddings_{timestamp}.csv"

    # Save the batch to a CSV file with timestamp
    save_batch_to_csv(batch_data, output_file_name)

    print(f"Processed batch of {len(file_keys)} files from {foldername}.")
    return

def process_all_files_in_folders(s3_client, bucket_name, folders, batch_size=100):
    """
    Iterate over all folders, process files in batches, and track progress.
    """
    total_files = 0
    processed_files = 0

    # Calculate the total number of .txt files in all folders
    for folder in folders:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        total_files += sum(1 for obj in response.get('Contents', []) if obj['Key'].endswith('.txt'))

    print(f"Total files to process: {total_files}")

    # Now process the files in batches
    for folder in folders:
        print(f"Processing files from folder: {folder}")
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        file_keys = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.txt')]

        # Process files in batches of 100
        for i in range(0, len(file_keys), batch_size):
            batch_file_keys = file_keys[i:i + batch_size]
            process_batch_of_papers(s3_client, bucket_name, batch_file_keys, folder)
            processed_files += len(batch_file_keys)

            # Update progress
            if processed_files % 100 == 0 or processed_files == total_files:
                print(f"Processed {processed_files}/{total_files} files.")

    print("Finished processing all files.")


In [106]:
process_all_files_in_folders(s3_client, bucket_name, folders[:3])

Total files to process: 2997
Processing files from folder: arxiv_papers/
processing batch of papers
creating batch
segmenting batch


KeyboardInterrupt: 

In [13]:
# Initialize S3 client
s3_client = boto3.client('s3')

bucket_name = 'arxivpapers'
folders = ['arxiv_papers/', 'conservapedia_papers/', 'vixra_papers/', 'wiki_papers/']

embedding_output_folder = 'embeddings/'  # Output folder for embeddings
model = SentenceTransformer("jordyvl/scibert_scivocab_uncased_sentence_transformer")

def save_to_csv(row, output_file_name):
    """
    Save a single row to a CSV file.
    """
    row_df = pd.DataFrame([row])  # Convert the row to a DataFrame
    row_df.to_csv(output_file_name, mode='a', index=False, header=not pd.io.common.file_exists(output_file_name))
    print(f"Saved to {output_file_name}")

def process_single_paper(s3_client, bucket_name, key, foldername):
    """
    Process a single paper: segment, embed, and save to CSV.
    """
    print(f"Processing file: {key}")

    # Stream file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=key)
    file_content = response['Body'].read().decode('utf-8')

    # Segmentation using SaT
    print("Segmenting paper")
    splits = list(sat_sm.split([file_content], remove_whitespace_before_inference=True, do_paragraph_segmentation=True))

    paragraphs_splits = []
    for paper in splits:
        paras = []
        for paragraph in paper:
            clean_sentences = [sentence.replace("\n", "") for sentence in paragraph]
            para = " ".join(clean_sentences)
            paras.append(para)
        paragraphs_splits.append(paras)

    # Embedding generation
    print("Generating embeddings")
    embeddings = model.encode(paragraphs_splits[0])  # Process the first (and only) paper in the batch

    # Prepare data for saving
    row = {
        'paper_embeddings': embeddings.tolist(),  # Convert embeddings to list for saving
        'source': foldername
    }

    # Generate timestamped filename for the CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_name = f"{embedding_output_folder}/embeddings_{timestamp}.csv"

    # Save to CSV
    save_to_csv(row, output_file_name)

def process_all_files_one_by_one(s3_client, bucket_name, folders):
    """
    Iterate over all folders and process files one by one.
    """
    total_files = 0
    processed_files = 0

    # Calculate the total number of .txt files in all folders
    for folder in folders:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        total_files += sum(1 for obj in response.get('Contents', []) if obj['Key'].endswith('.txt'))

    print(f"Total files to process: {total_files}")

    # Process files one by one
    for folder in folders:
        print(f"Processing files from folder: {folder}")
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)
        file_keys = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.txt')]

        for key in file_keys:
            process_single_paper(s3_client, bucket_name, key, folder)
            processed_files += 1

            print(f"Processed {processed_files}/{total_files} files.")

    print("Finished processing all files.")

# Run the script
#process_all_files_one_by_one(s3_client, bucket_name, folders)

In [11]:
folders[3:]

NameError: name 'folders' is not defined

In [15]:
# Run the script
import boto3
import pandas as pd
from sentence_transformers import SentenceTransformer
from datetime import datetime
process_all_files_one_by_one(s3_client, bucket_name, folders[3:])

Total files to process: 200
Processing files from folder: wiki_papers/
Processing file: wiki_papers/article_10000.txt
Segmenting paper
Generating embeddings
Saved to embeddings//embeddings_20241211_110605.csv
Processed 1/200 files.
Processing file: wiki_papers/article_10014.txt
Segmenting paper
Generating embeddings
Saved to embeddings//embeddings_20241211_110606.csv
Processed 2/200 files.
Processing file: wiki_papers/article_1019.txt
Segmenting paper
Generating embeddings
Saved to embeddings//embeddings_20241211_110608.csv
Processed 3/200 files.
Processing file: wiki_papers/article_10202.txt
Segmenting paper
Generating embeddings
Saved to embeddings//embeddings_20241211_110615.csv
Processed 4/200 files.
Processing file: wiki_papers/article_10216.txt
Segmenting paper
Generating embeddings
Saved to embeddings//embeddings_20241211_110618.csv
Processed 5/200 files.
Processing file: wiki_papers/article_1025.txt
Segmenting paper
Generating embeddings
Saved to embeddings//embeddings_20241211

KeyboardInterrupt: 

In [10]:
import os
import boto3

# Initialize S3 client
s3_client = boto3.client('s3')

# Configuration
s3_folder = 'wiki_papers/'
path = "/Users/skylershapiro/Downloads/paper-moderation-keti/wikipedia_articles"

# Get a list of .txt files in the directory
files = [f for f in os.listdir(path) if f.endswith('.txt')]

# Process and upload up to 200 files
for i, file_name in enumerate(files[:200]):
    local_file_path = os.path.join(path, file_name)
    s3_key = os.path.join(s3_folder, file_name)

    # Upload the file to S3
    try:
        s3_client.upload_file(local_file_path, bucket_name, s3_key)
        print(f"Uploaded {file_name} to S3: {s3_key}")
    except Exception as e:
        print(f"Failed to upload {file_name}: {e}")


Uploaded article_12415.txt to S3: wiki_papers/article_12415.txt
Uploaded article_6004.txt to S3: wiki_papers/article_6004.txt
Uploaded article_9337.txt to S3: wiki_papers/article_9337.txt
Uploaded article_8029.txt to S3: wiki_papers/article_8029.txt
Uploaded article_14064.txt to S3: wiki_papers/article_14064.txt
Uploaded article_10202.txt to S3: wiki_papers/article_10202.txt
Uploaded article_4613.txt to S3: wiki_papers/article_4613.txt
Uploaded article_18886.txt to S3: wiki_papers/article_18886.txt
Uploaded article_16673.txt to S3: wiki_papers/article_16673.txt
Uploaded article_2262.txt to S3: wiki_papers/article_2262.txt
Uploaded article_19540.txt to S3: wiki_papers/article_19540.txt
Uploaded article_16115.txt to S3: wiki_papers/article_16115.txt
Uploaded article_2504.txt to S3: wiki_papers/article_2504.txt
Uploaded article_19226.txt to S3: wiki_papers/article_19226.txt
Uploaded article_10564.txt to S3: wiki_papers/article_10564.txt
Uploaded article_4175.txt to S3: wiki_papers/article